# 상록원 메뉴 크롤링

---

In [19]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from apscheduler.schedulers.blocking import BlockingScheduler
from datetime import datetime
import re                                           
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime 

In [ ]:
def crawl():
    print(f"[{datetime.now()}] 크롤링 시작")
    

    BASE_URL  = "https://dgucoop.dongguk.edu/store/store.php"

    def fetch_week_page(offset):
        params = {"w": 4, "l": 2, "j": offset}
        resp = requests.get(BASE_URL, params=params)
        resp.encoding = 'euc-kr'
        return resp.text

    def clean_menu(raw):
        """메뉴 텍스트에서 불필요한 부분(괄호·가격·시간 등) 제거"""
        # 1) 괄호 안 내용 제거
        txt = re.sub(r'\(.*?\)', '', raw)
        # 2) 줄바꿈→공백
        txt = txt.replace('\n', ' ')
        # 3) 가격 패턴 제거
        txt = re.sub(r'￦\s*[0-9,]+|[0-9]+원', '', txt)
        # 4) 시간/기간 제거
        txt = re.sub(r'\d{1,2}:\d{2}(?:~\d{1,2}:\d{2})?', '', txt)
        # 5) 프로모션 별표 제거
        txt = re.sub(r'\*{2,}[^*]*', '', txt)
        # 6) 한글·공백 외 모두 제거
        txt = re.sub(r'[^가-힣\s&]', ' ', txt)
        # 7) 중복 공백 정리
        return re.sub(r'\s+', ' ', txt).strip()

    def clean_menu2(text):
        UNWANTED_WORDS = {
            '삼겹', '돼지', '오스트리아산', '닭', '브라질산',
            '소', '미국산', '스팸', '외국산', '돈가스',
            '낙지', '베트남산', '쌀', '배추김치', '배추', '고춧가루'
        }
        if not isinstance(text, str):
            return ''
        # 기존 정리 로직...
        text = re.sub(r'\(.*?\)', '', text)
        text = text.replace('\n', ' ')
        # text = re.sub(r'￦\s*[0-9,]+|[0-9]+원', '', text)
        text = re.sub(r'\d{1,2}:\d{2}(?:~\d{1,2}:\d{2})?(?:/?\s*\d{1,2}:\d{2}(?:~\d{1,2}:\d{2})?)?', '', text)
        text = re.sub(r'\*{2,}[^*]*', '', text)
        text = re.sub(r'[^가-힣\s&]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        for kw in ['전기실', '변압기', '고장', '운영중단', '분식당', '국내산']:
            if kw in text:
                return text.replace(f'{kw}', '')
        # 여기서 단어별로 분리한 뒤, UNWANTED_WORDS 에 딱 일치하는 토큰만 제거
        tokens = text.split()
        for tok in UNWANTED_WORDS:
            if tok in tokens:
                return text.replace(f'{tok}', '')

        return ' '.join(tokens)
    def extract_headers(table):
        """한 테이블의 요일+날짜 헤더( span태그 ) 리스트로 반환"""
        spans = table.select('tr.mft td span')
        return [span.get_text(strip=True) for span in spans]

    def parse_menu(html):
        soup    = BeautifulSoup(html, 'html.parser')
        records = []
        # 모든 메뉴 테이블 순회
        for table in soup.find_all('table', attrs={'bgcolor':'#CDD6B5'}):
            # 1) 헤더(요일+날짜) 추출
            headers = [span.get_text(strip=True) for span in table.select('tr.mft td span')]
            # 2) 실제 데이터는 세 번째 <tr> 부터
            rows = table.find_all('tr')[2:]
            current_restaurant = None
            current_category   = None

            for tr in rows:
                # — 식당(코너)명 만나면 갱신 —
                rst_td = tr.find('td', class_='menu_st')
                if rst_td:
                    current_restaurant = rst_td.get_text(strip=True)
                    current_category   = None
                    continue

                tds = tr.find_all('td')
                if not tds:
                    continue

                first = tds[0]
                # — 1) 일반 카테고리+중식 행 (rowspan=2, colspan 없음) —
                if first.has_attr('rowspan') and not first.has_attr('colspan'):
                    current_category = first.get_text(strip=True)
                    meal_type        = tds[1].get_text(strip=True)  # '중식'
                    start_idx        = 2

                # — 2) 특수 메뉴 블럭 (rowspan=7, colspan=2인 “메뉴”, “누리밥상” 등) —
                elif first.has_attr('rowspan') and first.has_attr('colspan'):
                    current_category = first.get_text(strip=True)
                    # colspan=2 이므로 tds[1]이 실제 첫날(일요일) 셀
                    # tds[1]이 비어 있으면, tds[2]가 첫날 메뉴일 수도 있으니 상황에 맞게 조정
                    meal_type = ''  # 필요시 비워두거나 '중식'으로 디폴트
                    start_idx = 1

                # — 3) 석식 행 (class="mft"만 있고 rowspan 없음) —
                elif 'mft' in first.get('class', []) and not first.has_attr('rowspan'):
                    meal_type = first.get_text(strip=True)  # '석식'
                    start_idx = 1

                else:
                    # 기타 불필요한 행(빈 행 등) 건너뛰기
                    continue

                # — 4) start_idx 에 맞춰 헤더와 메뉴 셀 매핑 —
                menu_cells = tds[start_idx:]
                for idx, cell in enumerate(menu_cells):
                    raw  = cell.get_text(' ', strip=True)
                    menu = clean_menu(raw)
                    if not menu:
                        continue
                    records.append({
                        "restaurant": current_restaurant,
                        "category":   current_category,
                        "meal":       meal_type,
                        "header":     headers[idx],  # headers 길이와 menu_cells 길이가 일치
                        "menu":       menu
                    })
                    
        df = pd.DataFrame(records)
        
        pattern = r'^([월화수목금토일])\s*(\d{1,2}월\s*\d{1,2}일)$'
        df[['week','date']] = df['header'].str.extract(pattern)
        df['menu_clean'] = df['menu'].apply(clean_menu2)
        df.drop(['header', 'menu'], axis=1, inplace=True)
        df['restaurant'][df['restaurant'].isna()] = '상록원3층식당'
        df[6:11]['category'].replace('석식', '집밥', inplace=True)
        df[6:11]['meal'].replace('', '석식', inplace=True)
        df.set_index('date', inplace=True)
        df.sort_index(inplace=True)
        return df

    def collect_since(start_date):
        offset    = 0
        all_weeks = []

        while True:
            html = fetch_week_page(offset)
            df   = parse_menu(html)

            # — 인덱스("05월 18일")에서 월·일 추출 —
            idx = df.index.to_series().astype(str)
            m_d = idx.str.extract(r'(\d{1,2})월\s*(\d{1,2})일')
            df['월'] = m_d[0].astype(int)
            df['일'] = m_d[1].astype(int)

            # — datetime 컬럼 생성 —
            df['date_dt'] = pd.to_datetime({
                'year':  start_date.year,
                'month': df['월'],
                'day':   df['일']
            })

            # 3월 1일 이전이면 종료
            if df['date_dt'].max().date() < start_date:
                break

            all_weeks.append(df)
            offset -= 1

        # 합치고, 3월 1일 이후만 필터
        full = pd.concat(all_weeks, ignore_index=False)
        return full[ full['date_dt'].dt.date >= start_date ]
    
    START = datetime.date(2025, 3, 1)
    menu_df = collect_since(START)
    result = menu_df[[
        'restaurant', 'category', 'meal', 'week', 'menu_clean'
    ]]
    result.to_csv('../data/DGU_menu_final.csv', encoding='utf-8-sig')

    print(f"[{datetime.now()}] 크롤링 완료")

if __name__ == "__main__":
    scheduler = BlockingScheduler(timezone="Asia/Seoul")

    # 매주 일요일 00:00에 실행
    scheduler.add_job(
        crawl,
        trigger='cron',
        day_of_week='sun',
        hour=0,
        minute=0,
        id='weekly_menu_crawl'
    )

    print("스케줄러 시작: 매주 일요일 00:00에 crawl() 실행")
    try:
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        print("스케줄러 종료") 

스케줄러 시작: 매주 일요일 00:00에 crawl() 실행


---